In [1]:
import numpy as np
from src.GEF import GEF
from Models.Classic import UpdateVals, TimeStep, Initialise
from scipy.integrate import solve_ivp

In [2]:
beta = 25
m = 6.16e-6
#Define Unitful Arrays and Functions
phi = 15.55
dphi = np.sqrt(2/3)*m
V = lambda x: 1/2*m**2*x**2
dV = lambda x: m**2*x

In [3]:
GNew = GEF("Classic", beta, {"phi":phi, "dphi":dphi}, {"V":V, "dV":dV})

/home/users/rfreiher/Documents/Axion_Inflation/WorkOnGEF/src/GEF.py:157: UserWarning: No default value set for 'kh'
  warnings.warn(f"No default value set for '{key}'")


In [5]:
GNew.dphi * GNew.phi

7.821057449390503e-05

In [ ]:
yini, vals = Initialise(GNew, 90)

In [ ]:
vallist = vals.ListValues()
for val in vallist:
    print(getattr(vals, val))

t (Unitless): 0.0
N (Unitless): 0.0
a (Unitless): 1.0
H (Unitless): None
phi (Unitless): 15.55
dphi (Unitless): 0.12844042552976437
ddphi (Unitless): None
E (Unitless): 0.0
B (Unitless): 0.0
G (Unitless): 0.0
xi (Unitless): None
kh (Unitless): 3.211010638244109


In [ ]:
def ode(t, y, vals, atol, rtol):
    UpdateVals(t, y, vals)
    dydt = TimeStep(t, y, vals, atol=atol, rtol=rtol)
    return dydt


In [ ]:
dydt = ode(0, yini, vals, 1e-20, 1e-6)

TypeError: '<' not supported between instances of 'BGVal' and 'int'